# <center> Environment Project</center>
#### <center> João Rodrigues 102487  &  Guilherme Casal 102587 </center>
### <center> Analysis of environment data </center>

In this work we will analyze various types of data related to pollution. We will try to obtain some conclusions, try to answer some question. To achieve this, the data will need treatment like nulls filling ,  outliers removal. In order to delevop this results, visualizing and observing the data will be key. 

For this project we are using the libraries _pandas_ for data management,  _seaborn_  and _matplotlib_for data visualization, _geopy_ to calculate distances between different locations and _skitlearn_ for the application of Machine Learning models

In [179]:
import pandas as pd
from sklearn.impute import KNNImputer
import seaborn as sns
from geopy.distance import geodesic
import matplotlib.pyplot as plt
from pandas.plotting import scatter_matrix

The data that we used was :  
- _Air Quality_ : It's a data-set from the city of London with the principal componentes of the air, which defines the quality of it. The data consists in  a mean of all days and it's grouped by month. Can be found at:  https://data.london.gov.uk/dataset

- _Pollution_ : This data-set contains the quantity of pollutant gases emited to the air per year. Many countries are included there.  
Can be found at:  https://www.eea.europa.eu/en/datahub/datahubitem-view/9405f714-8015-4b5b-a63c-280b82861b3d  

- _Traffic flow_ : Here we can find the traffic flow from different regions of London per year, distinguished for the  different roads of it.  
Can be found at: https://roadtraffic.dft.gov.uk/regions/6  

- _Health index_ : The last set of data represents the health index. Instead of of having a single index, it contains  one for each area . For exemaple, if one region has a lot of people that do exercise the health index will be postive in that area.  
Can be found at: https://www.ons.gov.uk/peoplepopulationandcommunity/healthandsocialcare/healthandwellbeing/datasets/healthindexunderlyingdataengland

### The first step is to load and filtrate the data for a area of 50 km of radius around the city of London

In [180]:
#Loading of the data

#air quality file
data_air = pd.read_csv("datasets/air-quality-london-monthly-averages.csv")

#pollution file
data_pol = pd.read_csv("datasets/pollution.csv",low_memory=False)

#traffic flow file
data_traf = pd.read_csv("datasets/dft_aadf_region_id_6.csv")

#health index file
data_health = pd.read_csv("datasets/healthindexunderlyingdataenglandcanscreenfix.csv",delimiter=";")

#### Before the filtering the data, we must visualize it on a table format to see all the variables and how their values are disposed 

The air and traffic flow data-set just includes the London city, thus there's no need to aplly this first filtering

In [181]:
data_air.head()

,Month,London Mean Roadside:Nitric Oxide (ug/m3),London Mean Roadside:Nitrogen Dioxide (ug/m3),London Mean Roadside:Oxides of Nitrogen (ug/m3),London Mean Roadside:Ozone (ug/m3),London Mean Roadside:PM10 Particulate (ug/m3),London Mean Roadside:PM2.5 Particulate (ug/m3),London Mean Roadside:Sulphur Dioxide (ug/m3),London Mean Background:Nitric Oxide (ug/m3),London Mean Background:Nitrogen Dioxide (ug/m3),London Mean Background:Oxides of Nitrogen (ug/m3),London Mean Background:Ozone (ug/m3),London Mean Background:PM10 Particulate (ug/m3),London Mean Background:PM2.5 Particulate (ug/m3),London Mean Background:Sulphur Dioxide (ug/m3)
0,Jan-08,NaN,55.5,NaN,29.5,25.0,14.7,4.2,NaN,42.3,NaN,36.9,18.8,.,3.6
1,Feb-08,NaN,75.9,NaN,20.3,39.5,28.8,7.6,NaN,60.2,NaN,26.4,31.9,.,6.7
2,Mar-08,NaN,55.6,NaN,40.1,21.6,12.3,3.9,NaN,39.8,NaN,50.2,15.5,.,2.3
3,Apr-08,NaN,61.8,NaN,37.9,28.7,20.5,4.5,NaN,44.0,NaN,50.1,21.7,.,3.2
4,May-08,NaN,62.9,NaN,46.3,34.6,27.5,4.6,NaN,44.1,NaN,60.5,29.5,16.6,4.3


In [182]:
data_traf.head()

,count_point_id,year,region_id,region_name,local_authority_id,local_authority_name,road_name,road_type,start_junction_road_name,end_junction_road_name,...,buses_and_coaches,lgvs,hgvs_2_rigid_axle,hgvs_3_rigid_axle,hgvs_4_or_more_rigid_axle,hgvs_3_or_4_articulated_axle,hgvs_5_articulated_axle,hgvs_6_articulated_axle,all_hgvs,all_motor_vehicles
0,6750,2014,6,London,96,Islington,A201,Major,Swinton St,Pentonville Rd,...,787,1728,204,74,132,15,6,18,449,11916
1,8352,2014,6,London,104,Lewisham,A2,Major,A20,A20,...,430,1415,258,36,58,6,11,22,390,9426
2,6807,2014,6,London,178,Kingston upon Thames,A240,Major,A3210,A307,...,175,1589,251,35,13,2,4,19,325,12951
3,27765,2014,6,London,57,Barnet,A5109,Major,Barnet Lane,A1000/A109,...,302,1651,246,12,12,0,1,1,273,19276
4,8488,2014,6,London,104,Lewisham,A20,Major,A20 Amersham Rd,A2 Parkfield Rd,...,656,1617,286,83,27,5,8,7,415,12821


In [183]:
data_pol.head()

,countryName,EPRTRSectorCode,eprtrSectorName,EPRTRAnnexIMainActivityCode,EPRTRAnnexIMainActivityLabel,FacilityInspireID,facilityName,facilityNameConfidentialityReason,Longitude,Latitude,addressConfidentialityReason,City,targetRelease,pollutant,emissions,reportingYear,releasesConfidentialityReason
0,Austria,4.0,Chemical industry,4(d),Chemical installations for the production on a...,AT.CAED/9008390392737.FACILITY,Nufarm GmbH & CO KG,NaN,14.336056,48.290943,NaN,Linz,AIR,"1,2-dichloroethane (DCE-1,2)",3300.0,2007,NaN
1,Austria,9.0,Other activities,9(c),Installations for the surface treatment of sub...,AT.CAED/9008390396414.FACILITY,MAGNA STEYR Fahrzeugtechnik AG&Co KG,NaN,15.477778,47.019170,NaN,"Graz,07.Bez.:Liebenau",AIR,Non-methane volatile organic compounds (NMVOC),170000.0,2007,NaN
2,Austria,6.0,Paper and wood production and processing,6(b),Industrial plants for the production of paper ...,AT.CAED/9008390397176.FACILITY,Fritz EGGER GmbH & Co. OG,NaN,15.691833,48.259556,NaN,Unterradlberg,AIR,Carbon dioxide (CO2),228000000.0,2007,NaN
3,Austria,6.0,Paper and wood production and processing,6(b),Industrial plants for the production of paper ...,AT.CAED/9008390397176.FACILITY,Fritz EGGER GmbH & Co. OG,NaN,15.691833,48.259556,NaN,Unterradlberg,AIR,Nitrogen oxides (NOX),311000.0,2007,NaN
4,Austria,3.0,Mineral industry,3(c),Installations for the production of cement cli...,AT.CAED/9008390397220.FACILITY,voestalpine Stahl GmbH,NaN,14.159778,47.807083,NaN,Klaus an der Pyhrnbahn,AIR,Carbon dioxide (CO2),329000000.0,2007,NaN


### Filtering cities

For the datasets that contain several cities in the United Kingdom, we created a function to find out all the cities contained in the dataset in question within a radius of 50 kilometers based on latitude and longitude.
This is because we think it is important to bring together cities closer together as they will certainly be the most impactful in terms of pollution.

In [184]:
def citiesWithinRadius(cities,radius,data_cities,data_from_outside=True):
    lat_lon_london = (51.509865, -0.118092)
    cidades = []
    #iterate all cities around london
    for city in cities:
        cidade = city
        #check if the data countains coordinates information or not and take the row with city iterated
        if data_from_outside:
            city = data_cities[data_cities["city"]== city]
        else:
            city = data_cities[data_cities["City"]== city]

        try:
            #take the corrdinates
            if data_from_outside:
                loc = (pd.array(city["lat"])[0],pd.array(city["lng"])[0])
            else:
                loc = (pd.array(city["Latitude"])[0],pd.array(city["Longitude"])[0])
        except:
            continue
        #if the distance is lower than 50, append to the list
        distance = geodesic(lat_lon_london, loc).kilometers
        if distance<=50:
            cidades.append(cidade)
    return cidades

Filtering of pollution data-set

In [185]:
#just Uk country
data_cities = data_pol[data_pol["countryName"]=="United Kingdom"]
#return all cities within a radius of 50 km from London
city_50km = citiesWithinRadius(data_health["Area name"].unique(), 50, data_cities,False)
#select the cities around london
data_pol = data_pol[data_pol["City"].isin(city_50km)]
data_pol.head()


,countryName,EPRTRSectorCode,eprtrSectorName,EPRTRAnnexIMainActivityCode,EPRTRAnnexIMainActivityLabel,FacilityInspireID,facilityName,facilityNameConfidentialityReason,Longitude,Latitude,addressConfidentialityReason,City,targetRelease,pollutant,emissions,reportingYear,releasesConfidentialityReason
289510,United Kingdom,5.0,Waste and wastewater management,5(d),Landfills (excluding landfills of inert waste ...,GB.EEA/13495.FACILITY,"Viridor Waste Management Ltd, Beddington Farml...",NaN,-0.142952,51.389667,NaN,Sutton,AIR,Chlorofluorocarbons (CFCs),11.10,2007,NaN
289511,United Kingdom,5.0,Waste and wastewater management,5(d),Landfills (excluding landfills of inert waste ...,GB.EEA/13495.FACILITY,"Viridor Waste Management Ltd, Beddington Farml...",NaN,-0.142952,51.389667,NaN,Sutton,AIR,Hydrochlorofluorocarbons (HCFCs),4.88,2007,NaN
289512,United Kingdom,5.0,Waste and wastewater management,5(d),Landfills (excluding landfills of inert waste ...,GB.EEA/13495.FACILITY,"Viridor Waste Management Ltd, Beddington Farml...",NaN,-0.142952,51.389667,NaN,Sutton,AIR,Methane (CH4),117000.00,2007,NaN
289806,United Kingdom,3.0,Mineral industry,3(e),"Installations for the manufacture of glass, in...",GB.EEA/31898.FACILITY,O-I Manufacturing UK,NaN,0.110430,51.783016,NaN,Harlow,AIR,Arsenic and compounds (as As),35.00,2007,NaN
289807,United Kingdom,3.0,Mineral industry,3(e),"Installations for the manufacture of glass, in...",GB.EEA/31898.FACILITY,O-I Manufacturing UK,NaN,0.110430,51.783016,NaN,Harlow,AIR,Nitrogen oxides (NOX),446000.00,2007,NaN


The health data-set doesn't include coordinates, so we have a complementary data-set that contains the coordinates of many cities around the world

In [186]:
data_health.head()


,Area code,Area name,Value,Numerator,Denominator,Indicator name,Year
0,E06000001,Hartlepool,"62,68947549",NaN,NaN,Physical activity,2015
1,E06000002,Middlesbrough,"62,8371136",NaN,NaN,Physical activity,2015
2,E06000003,Redcar and Cleveland,"65,80605787",NaN,NaN,Physical activity,2015
3,E06000004,Stockton-on-Tees,"59,83048525",NaN,NaN,Physical activity,2015
4,E06000005,Darlington,"65,39884288",NaN,NaN,Physical activity,2015


The data-set of the cities with their locations was found in : https://simplemaps.com/data/world-cities

Filtering of health index data-set

In [187]:
#information of the cities coordinates
data_cities = pd.read_csv("datasets/worldcities.csv")
data_cities = data_cities[data_cities["country"]=="United Kingdom"]
city_50km = citiesWithinRadius(data_health["Area name"].unique(), 50, data_cities)
data_health = data_health[data_health["Area name"].isin(city_50km)]
data_health.head()

### Not every variable is relevant to the analysis,thus some of them will be excluded  
Also, there will be a modification in the name of the columns, to simpler ones and a re-indexing, where the dates will be the "primary key", so, in the future, we are allowed to merge with other data-sets

Air quality is determined by the presence and concentration of various air pollutant, from those who influence air quality the most, three will be selected to the analysis.  
The components chosen have in consideration studies made in paper "Megacities and Atmospheric Pollution" from Mario J. Molina and Luisa T. Molina


In [ ]:
data_air.head()

In [ ]:
data_air.drop(["London Mean Roadside:Nitric Oxide (ug/m3)","London Mean Background:Nitric Oxide (ug/m3)",
               "London Mean Roadside:Oxides of Nitrogen (ug/m3)", "London Mean Background:Oxides of Nitrogen (ug/m3)"
               ],axis=1,inplace=True)
data_air.rename(columns={"London Mean Roadside:Nitrogen Dioxide (ug/m3)":"NO2 Air",
                         "London Mean Roadside:PM10 Particulate (ug/m3)":"PM10 Air",
                        "London Mean Roadside:PM2.5 Particulate (ug/m3)":"PM25",
                        "London Mean Roadside:Sulphur Dioxide (ug/m3)":"SO2 Air",
                        "London Mean Roadside:Ozone (ug/m3)" : "O3",
                        "London Mean Background:Nitrogen Dioxide (ug/m3)":"bg NO2",
                         "London Mean Background:PM10 Particulate (ug/m3)":"bg PM10",
                        "London Mean Background:PM2.5 Particulate (ug/m3)":"bg PM25",
                        "London Mean Background:Sulphur Dioxide (ug/m3)":"bg SO2",
                        "London Mean Background:Ozone (ug/m3)" : "bg O3",
                         },inplace=True)

data_air = data_air.loc[:,["Month","NO2 Air","PM10 Air","SO2 Air"]]

data_air.head()


In pollution data-set, the only relevant variables will be pollutant,the number of emissions, date, the city and the sector the emited the pollutant.  
In order to observe the evolution of each pulltant for each year, they will be tranformed to collums with pivot method


In [ ]:
data_pol.head()

In [ ]:
data_pol = data_pol[["pollutant","emissions","reportingYear"]]
data_pol.rename(columns={"reportingYear":"Year"},inplace=True)
data_pol = data_pol.groupby(["Year","pollutant"])["emissions"].mean().reset_index().rename(columns={"emissions":"emissions"})
data_pol = data_pol.pivot(index='Year', columns='pollutant', values='emissions')
data_pol.head()

We chose only a few pollutants because we had a lot of NaN values, that is, we chose based on the main pollutants that had the most impact on health and those that had the most values
Even so, we had some Nan Values and we decided to use a scatter matrix to see if we had some pollutants with similar values so that we could create a linear regression to fill in the values, but we still realized that it was not possible.

In [ ]:
data_pol = data_pol.loc[:, ["Ammonia (NH3)", "Carbon dioxide (CO2)",
                             "Methane (CH4)", "Nitrogen oxides (NOX)", "Arsenic and compounds (as As)", "Chlorofluorocarbons (CFCs)", "Hydrochlorofluorocarbons (HCFCs)", "Sulphur oxides (SOX)"]]
scatter_matrix(data_pol)
" "

Then we filter by the most relevant components emited to the atmosphere, according to the paper referenced before,  and fill them with the K-Nearest Neighbors

In [ ]:
data_pol = data_pol.loc[:, ["Ammonia (NH3)", "Sulphur oxides (SOX)",
                            "Methane (CH4)", "Nitrogen oxides (NOX)"]]

data_pol

The K-Nearest Neighbors (KNN) imputation works by finding the K nearest data points to the sample with missing values and imputing the missing values based on these nearby points. 

In [ ]:
# Reset the index of merged_df
def knn(data):
    index = data.index
    data.reset_index(drop=True, inplace=True)
    imputer = KNNImputer(n_neighbors=2)

    # Fill missing values using KNN
    merged_df = pd.DataFrame(imputer.fit_transform(data), columns=data.columns)

    merged_df = merged_df.set_index(index)
    return merged_df


data_pol = knn(data_pol)
data_pol


For this analysis we won't consider the flow in the different roads, thus the only variable related to that,  will be "all_motor_vehicles" which includes the sum of all road flows

In [ ]:
data_traf.head()

There will be only one value for each year, that corresponds to the sum of all regions of London selected previously

In [ ]:
data_traf = data_traf[["year","local_authority_name","all_motor_vehicles"]]
data_traf = data_traf.groupby('year')['all_motor_vehicles'].sum().reset_index()

data_traf =data_traf.set_index(["year"])
data_traf.head()

From the health index data-set will only be withdrawn, related to health, the value and the Indicator name(area which the index is being applied), since the numerator and denominator are only used to obtain the variable "value".

In [ ]:
data_health["Indicator name"].unique()

Just a few health factors will be selected. They are the ones we think have more correlation to pollution. They will also be now collumns of the data-set, so that we can have a row for each year
  

In [ ]:
data_health = data_health[["Area name","Value","Indicator name","Year"]].copy()
data_health['Value'] = data_health['Value'].str.replace(',', '.')
data_health["Value"] = data_health["Value"].astype(float)
data_health = data_health.groupby(["Year", "Indicator name"])["Value"].mean().reset_index().rename(columns={"Value":"Value"})
data_health = data_health.pivot(index='Year', columns='Indicator name', values='Value')
data_health= data_health.loc[:,["Air pollution", "Cancer", "Respiratory conditions"]]
data_health


We had values for all months of each year we decided to make an average per year.

In [ ]:
data_air["Month"] = pd.to_datetime(data_air["Month"], format='%b-%y').dt.year
data_air.rename(columns={"Month":"Year"}, inplace=True)
data_air = data_air.set_index("Year")
data_air = data_air.groupby("Year").mean()
data_air.head()


In the code below we verify NaN values for all datasets

In [ ]:
health_NaN = data_health.isna().any()
health_NaN

In [ ]:
air_NaN = data_air.isna().any()
air_NaN

In [ ]:
pol_NaN = data_pol.isna().any()
pol_NaN

In [ ]:
traf_NaN = data_traf.isna().any()
traf_NaN

### Plot some data


The points that the boxplot mark as outliers, dont seem like noisy data. Those points are just a slight deviation from the usual pattern, which is normal for envrionmental data, because this one is having some drastic changes in the last years ,so they will not be removed.


In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(12, 5))
for i, col in enumerate(data_health.columns):
    sns.boxplot(y=data_health[col], ax=axes[i])
    axes[i].set_title(f'Boxplot of {col}')

# Adjusting layout and displaying the figure
plt.tight_layout()
plt.show()

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(12, 5))
for i, col in enumerate(data_air.columns):
    sns.boxplot(y=data_air[col], ax=axes[i])
    axes[i].set_title(f'Boxplot of {col}')

# Adjusting layout and displaying the figure
plt.tight_layout()
plt.show()

In [ ]:
fig, axes = plt.subplots(1, 1, figsize=(12, 5))
for i, col in enumerate(data_traf.columns):
    sns.boxplot(y=data_traf[col], ax=axes)
    axes.set_title(f'Boxplot of {col}')

# Adjusting layout and displaying the figure
plt.tight_layout()
plt.show()

In [ ]:
fig, axes = plt.subplots(1, 4, figsize=(12, 5))
for i, col in enumerate(data_pol.columns):
    sns.boxplot(y=data_pol[col], ax=axes[i])
    axes[i].set_title(f'Boxplot of {col}')

# Adjusting layout and displaying the figure
plt.tight_layout()
plt.show()

### Merge Datasets 

Now all data will be merged, so that it can be analysed more easily in the future

In [ ]:
merged_df = data_traf.join(data_air,how="outer")
merged_df = merged_df.join(data_health,how="outer")
merged_df = merged_df.join(data_pol,how="outer")
merged_df.head(20)

The health index just have data since year 2015 until 2021, thus we will have two distinct  merged data-sets, in order to avoid large number of NaN values.

- The first one includes all data-sets since 2015 until 2021, also we will complete the Nan values with KNN

In [ ]:
merged_2015 = merged_df.loc[2015:2021,:]
merged_2015 


In [ ]:
merged_2015 = knn(merged_2015)
merged_2015

- The second one includes all data-sets except health index, also we will only merged data since 2007 until 2021, because traffic flow is the only one dataset that has data from 2007 backwards. The Nan values will be filled here, aswell.

In [ ]:
merged_no_health  = merged_df.drop(["Air pollution","Cancer","Respiratory conditions"],axis=1).loc[2007:2021,:]
merged_no_health


In [ ]:
merged_no_health = knn(merged_no_health)
merged_no_health